# Ajuste LOWESS para Tasa de Recuperación con Cálculo de RMSE y R²

Esta sección describe cómo aplicar LOWESS (también conocido como LOESS) para modelar la evolución de la tasa de recuperación (pago/saldo castigado) durante los 12 meses posteriores al castigo. Se asume que los datos están en forma de razón y se multiplican por 100 para expresarlos en porcentaje. Además, se calcularán las siguientes métricas:

- **Tendencia:** Definida como la derivada promedio de la curva suavizada, evaluada en el punto medio de los meses con datos válidos.
- **RMSE:** Raíz del Error Cuadrático Medio entre las predicciones LOWESS y los valores reales.
- **R²:** Coeficiente de determinación, calculado como  
  $$
  R^2 = 1 - \frac{\text{RSS}}{\text{TSS}},
  $$
  donde RSS es la suma de los cuadrados residuales y TSS es la suma total de los cuadrados respecto a la media.

> **Nota:**  
> Se analizarán únicamente aquellos meses en los que la tasa de recuperación es mayor a 0, para capturar la fase activa de recuperación. Los clientes que liquidan su deuda y luego muestran tasas de 0 no serán penalizados.

---

## Metodología Paso a Paso

1. **Extracción y Preprocesamiento:**
   - Se extraen los 12 valores correspondientes a la tasa de recuperación para cada cliente.
   - Se multiplican por 100 para expresarlos en porcentaje.
   - Se filtran los meses donde la tasa es mayor a 0, obteniendo así los datos válidos.

2. **Ajuste LOWESS:**
   - Con los datos válidos, se ajusta LOWESS utilizando la función `lowess` de `statsmodels.nonparametric`.
   - El parámetro `frac` controla la fracción de datos usados en cada ajuste local y, por tanto, el nivel de suavizado.

3. **Cálculo de la Derivada (Tendencia):**
   - Se calcula la derivada aproximada de la curva suavizada con `np.gradient`.
   - Se evalúa la derivada en el punto medio de los meses válidos para obtener la tendencia.

4. **Cálculo del RMSE y R²:**
   - Se generan las predicciones del ajuste LOWESS en los meses válidos.
   - Se calcula el RMSE comparando estas predicciones con los valores reales.
   - Se calcula R² a partir de la suma total de los cuadrados (TSS) y la suma de los cuadrados residuales (RSS).

5. **Almacenamiento de Resultados:**
   - Se añaden al DataFrame columnas para la tendencia LOWESS, el RMSE, el R² y el número de meses con datos válidos.

---

## Código Completo

```python
import pandas as pd
import numpy as np
import statsmodels.api as sm

def calcular_tendencia_recuperacion_lowess(df, frac=0.6):
    """
    Calcula la tendencia en la tasa de recuperación (pago/saldo castigado * 100)
    para cada cliente usando LOWESS, e incluye el cálculo del RMSE y del coeficiente de
    determinación (R²) del ajuste.
    
    Se asume que el DataFrame contiene las columnas:
      'tasa_recuperacion_mes1', 'tasa_recuperacion_mes2', ..., 'tasa_recuperacion_mes12'.
    
    Solo se consideran los meses en que la tasa de recuperación es mayor a 0.
    
    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame con 'cliente_id' y las columnas de tasa de recuperación.
    frac : float, opcional (default=0.6)
        Fracción de los datos a usar en cada ajuste local (controla el suavizado).
    
    Retorna:
    --------
    df : pd.DataFrame
        El DataFrame original con las siguientes columnas añadidas:
          - 'tendencia_recuperacion_lowess': Derivada evaluada en el punto medio (tendencia).
          - 'rmse_recuperacion_lowess': RMSE del ajuste LOWESS.
          - 'r2_recuperacion_lowess': Coeficiente de determinación (R²) del ajuste.
          - 'meses_con_datos': Número de meses con datos válidos (tasa > 0).
    """
    # Lista de columnas para la tasa de recuperación
    cols = [f'tasa_recuperacion_mes{i}' for i in range(1, 13)]
    
    # Inicializar las columnas de salida
    df["tendencia_recuperacion_lowess"] = np.nan
    df["rmse_recuperacion_lowess"] = np.nan
    df["r2_recuperacion_lowess"] = np.nan
    df["meses_con_datos"] = np.nan
    
    for idx, row in df.iterrows():
        # Extraer los valores y convertirlos a float, multiplicándolos por 100 para expresarlos en porcentaje.
        tasas = np.array([float(row[col]) * 100 for col in cols])
        
        # Filtrar los meses donde la tasa de recuperación es mayor a 0
        valid_idx = np.where(tasas > 0)[0]
        meses_validos = (valid_idx + 1).astype(float)  # Meses: 1, 2, ..., n
        tasas_validas = tasas[valid_idx]
        
        n_validos = len(meses_validos)
        df.at[idx, "meses_con_datos"] = n_validos
        
        if n_validos > 3:
            # Ajustar LOWESS a los datos válidos
            lowess_result = sm.nonparametric.lowess(tasas_validas, meses_validos, frac=frac, return_sorted=True)
            x_lowess = lowess_result[:, 0]
            y_lowess = lowess_result[:, 1]
            
            # Calcular la derivada aproximada de la curva LOWESS usando np.gradient
            derivada = np.gradient(y_lowess, x_lowess)
            # Evaluar la derivada en el punto medio
            x_mid = (meses_validos.min() + meses_validos.max()) / 2.0
            tendencia = np.interp(x_mid, x_lowess, derivada)
            
            # Predicciones LOWESS en los meses válidos
            y_pred = np.interp(meses_validos, x_lowess, y_lowess)
            # Calcular RMSE
            rmse = np.sqrt(np.mean((tasas_validas - y_pred) ** 2))
            
            # Calcular R²:
            TSS = np.sum((tasas_validas - np.mean(tasas_validas)) ** 2)
            RSS = np.sum((tasas_validas - y_pred) ** 2)
            r2 = 1 - (RSS / TSS) if TSS > 0 else np.nan
        else:
            tendencia = np.nan
            rmse = np.nan
            r2 = np.nan
        
        df.at[idx, "tendencia_recuperacion_lowess"] = tendencia
        df.at[idx, "rmse_recuperacion_lowess"] = rmse
        df.at[idx, "r2_recuperacion_lowess"] = r2
        
    return df

# Ejemplo de uso:
df = pd.read_csv("datos_recuperacion.csv")  # El archivo debe contener 'cliente_id' y 'tasa_recuperacion_mes1' a 'tasa_recuperacion_mes12'
df_actualizado_lowess = calcular_tendencia_recuperacion_lowess(df, frac=0.6)
df_actualizado_lowess.to_csv("datos_recuperacion_actualizados_lowess.csv", index=False)
print(df_actualizado_lowess.head())


# Ajuste Polinómico para Tasa de Recuperación con Selección Automática del Grado, RMSE y R²

Esta sección describe cómo aplicar una regresión polinómica con selección automática del grado óptimo para modelar la evolución de la tasa de recuperación (pago/saldo castigado) durante los 12 meses posteriores al castigo. Se asume que los datos originales son ratios y se multiplican por 100 para expresarlos en porcentaje. Además, se calculan las siguientes métricas:

- **Tendencia:** Se define como la derivada del polinomio evaluada en el punto medio de los meses con datos válidos (aquellos en que la tasa es mayor a 0).
- **RMSE:** La raíz del error cuadrático medio entre las predicciones del modelo y los valores reales.
- **R²:** El coeficiente de determinación, calculado como  
  $$
  R^2 = 1 - \frac{\text{RSS}}{\text{TSS}},
  $$
  donde RSS es la suma de los cuadrados residuales y TSS es la suma total de los cuadrados respecto a la media.

Se analizan únicamente los meses en que la tasa de recuperación es mayor a 0, para capturar la fase activa de recuperación. Aquellos clientes que liquiden su deuda (y luego muestren tasas de 0) no serán penalizados, ya que se analiza solo la parte activa.

---

## Metodología Paso a Paso

1. **Extracción y Preprocesamiento:**
   - Se extraen los 12 valores correspondientes a la tasa de recuperación para cada cliente.
   - Cada valor se multiplica por 100 para expresarlo en porcentaje.
   - Se filtran los meses en que la tasa es mayor a 0, obteniendo así el conjunto de datos válidos para el análisis.

2. **Selección del Modelo Polinómico:**
   - Se ajusta un polinomio a los datos válidos para diferentes grados, desde 1 hasta el mínimo entre un grado máximo predefinido (`max_degree`) y (número de datos válidos – 1).
   - Para cada grado se calcula el AIC:
     
     $$
     \text{AIC} = n \cdot \ln\left(\frac{\text{RSS}}{n}\right) + 2 \cdot (grado + 1),
     $$
     
     donde \( n \) es el número de datos válidos y RSS es la suma de los cuadrados de los residuos.
   - Se selecciona el grado que minimice el AIC.

3. **Cálculo de la Tendencia, RMSE y R²:**
   - Con el modelo óptimo se calcula la derivada del polinomio y se evalúa en el punto medio de los meses válidos para obtener la tendencia.
   - Se calculan las predicciones del modelo en los mismos puntos para obtener el RMSE.
   - Se calcula R² utilizando la fórmula:
     
     $$
     R^2 = 1 - \frac{\text{RSS}}{\text{TSS}},
     $$
     
     donde TSS es la suma total de los cuadrados respecto a la media de los datos válidos.

4. **Almacenamiento de Resultados:**
   - Se añaden nuevas columnas al DataFrame para almacenar:
     - La tendencia del polinomio (`tendencia_recuperacion_polinomica`).
     - El grado óptimo seleccionado (`grado_optimo_recuperacion`).
     - El RMSE (`rmse_recuperacion_polinomica`).
     - El coeficiente de determinación R² (`r2_recuperacion_polinomica`).
     - El número de meses con datos válidos (`meses_con_datos_recuperacion`).

---

## Código Completo

```python
import pandas as pd
import numpy as np

def calcular_tendencia_recuperacion_polinomica_variable(df, max_degree=5, epsilon=1e-8):
    """
    Calcula la tendencia en la tasa de recuperación (pago/saldo castigado * 100)
    para cada cliente usando regresión polinómica con selección automática del grado óptimo.
    Además, calcula el RMSE y el coeficiente de determinación (R²) del ajuste.
    
    Se asume que el DataFrame contiene 'cliente_id' y las columnas:
      'tasa_recuperacion_mes1', 'tasa_recuperacion_mes2', ..., 'tasa_recuperacion_mes12'.
    
    Solo se consideran los meses en que la tasa de recuperación es mayor a 0.
    
    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame con las columnas mencionadas.
    max_degree : int, opcional (default=5)
        Grado máximo a considerar para el polinomio.
    epsilon : float, opcional (default=1e-8)
        Valor pequeño para evitar problemas en el cálculo del AIC.
    
    Retorna:
    --------
    df : pd.DataFrame
        El DataFrame original con las nuevas columnas:
          - 'tendencia_recuperacion_polinomica': Derivada evaluada en el punto medio (medida de la tendencia).
          - 'grado_optimo_recuperacion': Grado del polinomio seleccionado.
          - 'rmse_recuperacion_polinomica': RMSE del ajuste.
          - 'r2_recuperacion_polinomica': Coeficiente de determinación (R²) del ajuste.
          - 'meses_con_datos_recuperacion': Número de meses con tasa de recuperación > 0.
    """
    # Lista de columnas de tasa de recuperación
    cols = [f'tasa_recuperacion_mes{i}' for i in range(1, 13)]
    
    # Inicializar las columnas de salida
    df["tendencia_recuperacion_polinomica"] = np.nan
    df["grado_optimo_recuperacion"] = np.nan
    df["rmse_recuperacion_polinomica"] = np.nan
    df["r2_recuperacion_polinomica"] = np.nan
    df["meses_con_datos_recuperacion"] = np.nan

    for idx, row in df.iterrows():
        # Extraer los valores y convertirlos a float, multiplicándolos por 100 para expresarlos en porcentaje.
        tasas = np.array([float(row[col]) * 100 for col in cols])
        
        # Filtrar los meses donde la tasa de recuperación es mayor a 0
        valid_idx = np.where(tasas > 0)[0]
        meses_validos = (valid_idx + 1).astype(float)  # Meses: 1, 2, ..., n
        tasas_validas = tasas[valid_idx]
        
        n_points = len(meses_validos)
        df.at[idx, "meses_con_datos_recuperacion"] = n_points
        
        # Si no hay suficientes datos, asignar NaN a las métricas
        if n_points < 2:
            df.at[idx, "tendencia_recuperacion_polinomica"] = np.nan
            df.at[idx, "grado_optimo_recuperacion"] = np.nan
            df.at[idx, "rmse_recuperacion_polinomica"] = np.nan
            df.at[idx, "r2_recuperacion_polinomica"] = np.nan
            continue
        
        # Definir el rango de grados a considerar: de 1 hasta min(max_degree, n_points - 1)
        max_deg = min(max_degree, n_points - 1)
        best_AIC = np.inf
        best_degree = None
        best_coeffs = None
        
        # Iterar sobre grados candidatos
        for degree in range(1, max_deg + 1):
            try:
                coeffs = np.polyfit(meses_validos, tasas_validas, deg=degree)
                y_pred = np.polyval(coeffs, meses_validos)
                RSS = np.sum((tasas_validas - y_pred) ** 2)
                RSS = max(RSS, epsilon)  # Evitar log(0)
                k = degree + 1
                AIC = n_points * np.log(RSS / n_points) + 2 * k
            except Exception as e:
                continue
            
            if AIC < best_AIC:
                best_AIC = AIC
                best_degree = degree
                best_coeffs = coeffs
        
        if best_coeffs is not None:
            # Calcular la derivada del polinomio ajustado
            d_coeffs = np.polyder(best_coeffs)
            x_mid = (meses_validos.min() + meses_validos.max()) / 2.0
            tendencia = np.polyval(d_coeffs, x_mid)
            
            # Calcular las predicciones del modelo en los puntos válidos
            y_pred = np.polyval(best_coeffs, meses_validos)
            rmse = np.sqrt(np.mean((tasas_validas - y_pred) ** 2))
            
            TSS = np.sum((tasas_validas - np.mean(tasas_validas)) ** 2)
            RSS = np.sum((tasas_validas - y_pred) ** 2)
            r2 = 1 - (RSS / TSS) if TSS > 0 else np.nan
        else:
            tendencia = np.nan
            best_degree = np.nan
            rmse = np.nan
            r2 = np.nan
        
        df.at[idx, "tendencia_recuperacion_polinomica"] = tendencia
        df.at[idx, "grado_optimo_recuperacion"] = best_degree
        df.at[idx, "rmse_recuperacion_polinomica"] = rmse
        df.at[idx, "r2_recuperacion_polinomica"] = r2

    return df

# Ejemplo de uso:
df = pd.read_csv("datos_recuperacion.csv")  # Se asume que el DataFrame contiene 'cliente_id' y 'tasa_recuperacion_mes1' a 'tasa_recuperacion_mes12'
df_actualizado_recuperacion_poly = calcular_tendencia_recuperacion_polinomica_variable(df, max_degree=5)
df_actualizado_recuperacion_poly.to_csv("datos_recuperacion_actualizados_polinomica.csv", index=False)
print(df_actualizado_recuperacion_poly.head())


# Ajuste Spline Suavizante para Tasa de Recuperación con RMSE y R²

Esta sección replica el ejercicio anterior, pero ahora aplicado a la tasa de recuperación (pago/saldo castigado). Se analiza la evolución mes a mes durante los 12 meses posteriores al castigo. Se utiliza un spline suavizante para modelar la serie de tiempo de la tasa de recuperación, y se calculan además el RMSE y el coeficiente de determinación (R²) del ajuste.

> **Nota sobre el comportamiento:**  
> En muchos casos, después del castigo, los clientes no realizan pagos y la tasa de recuperación es 0. Sin embargo, algunos clientes muestran tasas de recuperación positivas durante un período, y en ciertos casos la tasa es alta hasta que se paga por completo la deuda, momento en el que la tasa cae a 0. Este último comportamiento no debe interpretarse como negativo, ya que indica que el cliente ha liquidado su deuda.  
>  
> Para capturar adecuadamente este comportamiento, se recomienda analizar únicamente los meses en que la tasa de recuperación es mayor a 0 (fase de recuperación activa). Otra alternativa es trabajar con la recuperación acumulada, pero aquí replicamos el mismo enfoque de spline suavizante aplicado a la tasa mensual.

---

## Metodología Paso a Paso

1. **Extracción y Preprocesamiento:**
   - Se extraen los 12 valores correspondientes a la tasa de recuperación para cada cliente.  
   - Se convierten a porcentaje (multiplicando por 100) si los datos están en forma de razón.  
   - Se filtran aquellos meses en que la tasa de recuperación es mayor a 0, para analizar únicamente la fase en la que se observa actividad.

2. **Ajuste del Spline Suavizante:**
   - Se define el conjunto de puntos \((x, y)\), donde \(x\) es el número del mes (1, 2, ..., 12) y \(y\) es la tasa de recuperación (en porcentaje).  
   - Se utiliza `UnivariateSpline` de `scipy.interpolate` para ajustar una curva suave a los datos válidos.  
   - El parámetro `s_factor` controla el nivel de suavizado.

3. **Cálculo de la Derivada (Tendencia):**
   - Se calcula la derivada del spline usando `spline.derivative()`.  
   - La tendencia se estima evaluando esta derivada en el punto medio de los meses válidos:
     $$
     x_{\text{mid}} = \frac{\min(x) + \max(x)}{2}.
     $$
   - Una derivada positiva indica que la tasa de recuperación está en aumento, lo cual es un buen indicio, mientras que una derivada negativa indicaría lo contrario.

4. **Cálculo del RMSE y R²:**
   - Se obtienen las predicciones del spline para los meses válidos.
   - **RMSE:**  
     Se calcula como:
     $$
     \text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(y_i - \hat{y}_i\right)^2},
     $$
     donde \(y_i\) son los valores reales y \(\hat{y}_i\) las predicciones.
   - **R²:**  
     Se calcula usando:
     $$
     R^2 = 1 - \frac{\text{RSS}}{\text{TSS}},
     $$
     donde:
     - **RSS** es la suma de los cuadrados residuales \(\sum (y_i - \hat{y}_i)^2\).
     - **TSS** es la suma total de los cuadrados \(\sum (y_i - \bar{y})^2\).

5. **Almacenamiento de Resultados:**
   - Se añaden al DataFrame las columnas con:
     - La tendencia del spline (`tendencia_recuperacion_spline`).
     - El RMSE (`rmse_recuperacion_spline`).
     - El R² (`r2_recuperacion_spline`).
     - El número de meses con datos válidos (`meses_con_datos`).

---

## Código Completo

```python
import pandas as pd
import numpy as np
from scipy.interpolate import UnivariateSpline

def calcular_tendencia_recuperacion_spline(df, s_factor=1.0):
    """
    Calcula la tendencia en la tasa de recuperación (pago/saldo castigado * 100)
    para cada cliente usando un spline suavizante (UnivariateSpline), e incluye el cálculo
    del RMSE y del coeficiente de determinación (R²) del ajuste.
    
    Se asume que el DataFrame contiene las columnas:
      'tasa_recuperacion_mes1', 'tasa_recuperacion_mes2', ..., 'tasa_recuperacion_mes12'.
    
    Nota: Muchos clientes tendrán tasas de recuperación 0. Se analizarán únicamente aquellos meses
          en que la tasa es mayor a 0 para capturar la fase de recuperación activa.
    
    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame que contiene 'cliente_id' y las columnas mencionadas.
    s_factor : float, opcional (default=1.0)
        Factor de suavizado para el spline.
    
    Retorna:
    --------
    df : pd.DataFrame
        El DataFrame original con las siguientes columnas añadidas:
          - 'tendencia_recuperacion_spline': Derivada del spline evaluada en el punto medio.
          - 'rmse_recuperacion_spline': RMSE del ajuste del spline.
          - 'r2_recuperacion_spline': Coeficiente de determinación (R²) del ajuste.
          - 'meses_con_datos': Número de meses con datos válidos (tasa de recuperación > 0).
    """
    # Lista de columnas de tasa de recuperación
    cols = [f'tasa_recuperacion_mes{i}' for i in range(1, 13)]
    
    # Inicializar las columnas de salida
    df['tendencia_recuperacion_spline'] = np.nan
    df['rmse_recuperacion_spline'] = np.nan
    df['r2_recuperacion_spline'] = np.nan
    df['meses_con_datos'] = np.nan
    
    for idx, row in df.iterrows():
        # Extraer los valores y convertirlos a float. Se asume que los valores están en forma de ratio,
        # por lo que se multiplican por 100 para expresarlos como porcentaje.
        recuperacion = np.array([float(row[col]) * 100 for col in cols])
        
        # Filtrar los meses donde la tasa de recuperación es mayor a 0 (fase de recuperación activa)
        idx_validos = np.where(recuperacion > 0)[0]
        meses_validos = (idx_validos + 1).astype(float)
        recuperacion_validas = recuperacion[idx_validos]
        
        n_validos = len(meses_validos)
        df.at[idx, 'meses_con_datos'] = n_validos
        
        if n_validos > 3:
            # Ajustar el spline suavizante a los datos válidos
            spline = UnivariateSpline(meses_validos, recuperacion_validas, s=s_factor)
            # Calcular el punto medio de los meses válidos
            x_mid = (meses_validos.min() + meses_validos.max()) / 2.0
            # Evaluar la derivada del spline en el punto medio (tendencia)
            tendencia = spline.derivative()(x_mid)
            
            # Obtener las predicciones del spline en los meses válidos
            y_pred = spline(meses_validos)
            # Calcular el RMSE
            rmse = np.sqrt(np.mean((recuperacion_validas - y_pred) ** 2))
            
            # Calcular R²:
            TSS = np.sum((recuperacion_validas - np.mean(recuperacion_validas)) ** 2)
            RSS = np.sum((recuperacion_validas - y_pred) ** 2)
            r2 = 1 - (RSS / TSS) if TSS > 0 else np.nan
        else:
            tendencia = np.nan
            rmse = np.nan
            r2 = np.nan
        
        df.at[idx, 'tendencia_recuperacion_spline'] = tendencia
        df.at[idx, 'rmse_recuperacion_spline'] = rmse
        df.at[idx, 'r2_recuperacion_spline'] = r2
        
    return df

# Ejemplo de uso:
df = pd.read_csv("datos_recuperacion.csv")  # Se asume que el archivo contiene 'cliente_id' y 'tasa_recuperacion_mes1' a 'tasa_recuperacion_mes12'
df_actualizado_recuperacion = calcular_tendencia_recuperacion_spline(df, s_factor=1.0)
df_actualizado_recuperacion.to_csv("datos_recuperacion_actualizados_spline.csv", index=False)
print(df_actualizado_recuperacion.head())
